# 掩码语言模型

In [1]:
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForMaskedLM, DataCollatorForLanguageModeling, TrainingArguments, Trainer

In [3]:
ds = Dataset.load_from_disk("../data/wiki_cn_filtered")
ds[0]

{'source': 'wikipedia.zh2307',
 'completion': "西安交通大学博物馆（Xi'an Jiaotong University Museum）是一座位于西安交通大学的博物馆，馆长是锺明善。\n历史\n2004年9月20日开始筹建，2013年4月8日正式建成开馆，位于西安交通大学兴庆校区陕西省西安市咸宁西路28号。建筑面积6,800平米，展厅面积4,500平米，馆藏文物4,900余件。包括历代艺术文物馆、碑石书法馆、西部农民画馆、邢良坤陶瓷艺术馆、陕西秦腔博物馆和书画展厅共五馆一厅。\n营业时间\n* 周一至周六：上午九点至十二点，下午一点至五点\n* 周日闭馆"}

In [4]:
tokenizer = AutoTokenizer.from_pretrained("../hfl/chinese-macbert-base")

def process_func(examples):
    return tokenizer(examples["completion"], max_length=384, truncation=True)

tokenized_ds = ds.map(process_func, batched=True, remove_columns=ds.column_names)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [5]:
tokenized_ds

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 10000
})

In [6]:
from torch.utils.data import DataLoader

dl = DataLoader(tokenized_ds, batch_size=2, collate_fn=DataCollatorForLanguageModeling(tokenizer, mlm_probability=0.15))

In [7]:
next(enumerate(dl))

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


(0,
 {'input_ids': tensor([[  101,  6205,  2128,   769,   103,  1920,  2110,  1300,  4289,   103,
           8020, 13135,   112,  9064, 12095,  8731,  8626,  8181,  8736, 10553,
            103,  3221,   671,  2429,   855,   754,  6205,  2128,   769,   103,
           1920,  2110,  4638,  1300,  4289,  7667,  8024,  7667,  7270,  3221,
           7247,  4964,  1587,   511,  1325,  1380,  8258,   103,   130,  3299,
           8113,  3189,  2458,  1993,  5040,  2456,  8024,  8138,  2399,   125,
           3299,   129,   103,  3633,   103,  2456,  2768,  2458,  7667,  8024,
            103,   754,  6205,  2128,   769,  6858,  1920,  2110,   103,  2412,
           3413,  1277,   103,   103,  4689,  6205,  2128,  2356,  1496,  2123,
           6205,  6662,  8143,  1384,   511,  2456,  5029,  7481,   103,   127,
            117,  8280,  2398,   103,  8024,  2245,  1324,  7481,  4916,   125,
            117,  8195,  2398,  5101,  8024,  7667,   103,  3152,  4289,   125,
            117,  8567

In [8]:
tokenizer.mask_token, tokenizer.mask_token_id

('[MASK]', 103)

In [10]:
model = AutoModelForMaskedLM.from_pretrained("../hfl/chinese-macbert-base")

d:\Programs\miniconda3\envs\llm\lib\site-packages\transformers\modeling_utils.py:442: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(checkpoint_file, map_lo

In [11]:
args = TrainingArguments(
    output_dir="./mask_lm",
    per_device_train_batch_size=32,
    logging_steps=10,
    num_train_epochs=1
)

In [12]:
trainer = Trainer(
    args=args,
    model=model,
    tokenizer=tokenizer,
    train_dataset=tokenized_ds,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=True, mlm_probability=0.15)
)

In [13]:
trainer.train()

d:\Programs\miniconda3\envs\llm\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/313 [00:00<?, ?it/s]

{'loss': 1.4291, 'learning_rate': 4.840255591054313e-05, 'epoch': 0.03}
{'loss': 1.407, 'learning_rate': 4.680511182108626e-05, 'epoch': 0.06}
{'loss': 1.337, 'learning_rate': 4.520766773162939e-05, 'epoch': 0.1}
{'loss': 1.4255, 'learning_rate': 4.361022364217253e-05, 'epoch': 0.13}
{'loss': 1.3411, 'learning_rate': 4.201277955271566e-05, 'epoch': 0.16}
{'loss': 1.4069, 'learning_rate': 4.041533546325879e-05, 'epoch': 0.19}
{'loss': 1.3311, 'learning_rate': 3.8817891373801916e-05, 'epoch': 0.22}
{'loss': 1.3319, 'learning_rate': 3.722044728434505e-05, 'epoch': 0.26}
{'loss': 1.3291, 'learning_rate': 3.562300319488818e-05, 'epoch': 0.29}
{'loss': 1.3182, 'learning_rate': 3.402555910543131e-05, 'epoch': 0.32}
{'loss': 1.2867, 'learning_rate': 3.242811501597444e-05, 'epoch': 0.35}
{'loss': 1.3335, 'learning_rate': 3.083067092651757e-05, 'epoch': 0.38}
{'loss': 1.3033, 'learning_rate': 2.9233226837060707e-05, 'epoch': 0.42}
{'loss': 1.3037, 'learning_rate': 2.7635782747603834e-05, 'epoch'

TrainOutput(global_step=313, training_loss=1.3293065598216682, metrics={'train_runtime': 6580.3686, 'train_samples_per_second': 1.52, 'train_steps_per_second': 0.048, 'train_loss': 1.3293065598216682, 'epoch': 1.0})

In [14]:
from transformers import pipeline

pipe = pipeline("fill-mask", model=model, tokenizer=tokenizer, device=0)

In [15]:
pipe("西安交通[MASK][MASK]博物馆（Xi'an Jiaotong University Museum）是一座位于西安交通大学的博物馆")

[[{'score': 0.9977681636810303,
   'token': 1920,
   'token_str': '大',
   'sequence': "[CLS] 西 安 交 通 大 [MASK] 博 物 馆 （ xi'an jiaotong university museum ） 是 一 座 位 于 西 安 交 通 大 学 的 博 物 馆 [SEP]"},
  {'score': 0.0014151427894830704,
   'token': 2110,
   'token_str': '学',
   'sequence': "[CLS] 西 安 交 通 学 [MASK] 博 物 馆 （ xi'an jiaotong university museum ） 是 一 座 位 于 西 安 交 通 大 学 的 博 物 馆 [SEP]"},
  {'score': 8.18535772850737e-05,
   'token': 4906,
   'token_str': '科',
   'sequence': "[CLS] 西 安 交 通 科 [MASK] 博 物 馆 （ xi'an jiaotong university museum ） 是 一 座 位 于 西 安 交 通 大 学 的 博 物 馆 [SEP]"},
  {'score': 5.6669607147341594e-05,
   'token': 7770,
   'token_str': '高',
   'sequence': "[CLS] 西 安 交 通 高 [MASK] 博 物 馆 （ xi'an jiaotong university museum ） 是 一 座 位 于 西 安 交 通 大 学 的 博 物 馆 [SEP]"},
  {'score': 5.5355747463181615e-05,
   'token': 2339,
   'token_str': '工',
   'sequence': "[CLS] 西 安 交 通 工 [MASK] 博 物 馆 （ xi'an jiaotong university museum ） 是 一 座 位 于 西 安 交 通 大 学 的 博 物 馆 [SEP]"}],
 [{'score': 0.998867392539

In [16]:
pipe("下面是一则[MASK][MASK]新闻。小编报道，近日，游戏产业发展的非常好！")

[[{'score': 0.11592137068510056,
   'token': 7028,
   'token_str': '重',
   'sequence': '[CLS] 下 面 是 一 则 重 [MASK] 新 闻 。 小 编 报 道 ， 近 日 ， 游 戏 产 业 发 展 的 非 常 好 ！ [SEP]'},
  {'score': 0.05849061161279678,
   'token': 4178,
   'token_str': '热',
   'sequence': '[CLS] 下 面 是 一 则 热 [MASK] 新 闻 。 小 编 报 道 ， 近 日 ， 游 戏 产 业 发 展 的 非 常 好 ！ [SEP]'},
  {'score': 0.05292018875479698,
   'token': 2031,
   'token_str': '娱',
   'sequence': '[CLS] 下 面 是 一 则 娱 [MASK] 新 闻 。 小 编 报 道 ， 近 日 ， 游 戏 产 业 发 展 的 非 常 好 ！ [SEP]'},
  {'score': 0.04742466285824776,
   'token': 3952,
   'token_str': '游',
   'sequence': '[CLS] 下 面 是 一 则 游 [MASK] 新 闻 。 小 编 报 道 ， 近 日 ， 游 戏 产 业 发 展 的 非 常 好 ！ [SEP]'},
  {'score': 0.04014512896537781,
   'token': 3173,
   'token_str': '新',
   'sequence': '[CLS] 下 面 是 一 则 新 [MASK] 新 闻 。 小 编 报 道 ， 近 日 ， 游 戏 产 业 发 展 的 非 常 好 ！ [SEP]'}],
 [{'score': 0.08176324516534805,
   'token': 4829,
   'token_str': '磅',
   'sequence': '[CLS] 下 面 是 一 则 [MASK] 磅 新 闻 。 小 编 报 道 ， 近 日 ， 游 戏 产 业 发 展 的 非 常 好 ！ [SEP]'},
  {